In [36]:
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook
from random import choice
from string import ascii_lowercase
import re

In [37]:
wb = Workbook()
ws1 = wb.create_sheet('Giełda')
ws2 = wb.create_sheet('Linki')
ws3 = wb.create_sheet('Filmweb')

In [38]:
def generateString(n):
    return "".join(choice(ascii_lowercase) for i in range(n))
codes = [generateString(3) for _ in range(5)]
s = requests.Session()
valid_codes = []
for code in codes:
    print(code)
    params = {"s": code}
    response = requests.get("http://stooq.pl/q/", params=params)
    soup = BeautifulSoup(response.content, 'html.parser')
    if len(soup.body.findAll('span', id='c2')) != 0:
        link_content = soup.find('font',id='f16').find('a',href=True)
        if link_content.text == 'symbol waloru':
            print("Kod ani zaden podobny jemu nie zostal znaleziony.")
        else:
            valid_code = link_content['href'][5:]
            valid_codes.append(valid_code)
            print(f'Znaleziono kod podobny: {valid_code}')
    else:
        valid_codes.append(code)

        
ws1.cell(row=1, column=1).value = 'Kod'
ws1.cell(row=1, column=2).value = 'Kurs'
ws1.cell(row=1, column=3).value = 'Zmiana procentowa'
ws1.cell(row=1, column=4).value = 'Transakcje'
row = 2
for code in valid_codes:
    print(code)
    params = {"s": code}
    response = requests.get("http://stooq.pl/q/", params=params)
    soup = BeautifulSoup(response.content, 'html.parser')
    price = soup.find(text="Kurs").parent.find("span").text
    percent_change = soup.find(text="Zmiana").parent.findAll("span")[1].text
    transactions = soup.find(text="Transakcje").parent.find("span").text
    print(f"{price}, {percent_change}, {transactions}")
    ws1.cell(row=row, column=1).value = code
    ws1.cell(row=row, column=2).value = price if price else 'Brak danych'
    ws1.cell(row=row, column=3).value =  percent_change if percent_change else 'Brak danych'
    ws1.cell(row=row, column=4).value =  transactions if transactions else 'Brak danych'
    row += 1

nfb
Znaleziono kod podobny: nfbk.us
jzo
Kod ani zaden podobny jemu nie zostal znaleziony.
nhh
Kod ani zaden podobny jemu nie zostal znaleziony.
wfk
Kod ani zaden podobny jemu nie zostal znaleziony.
geo
Znaleziono kod podobny: geo.uk
nfbk.us
17.11, (+1.12%), 539
geo.uk
1.375, (0.00%), 1


In [39]:
url = "https://www.baeldung.com/"
req = requests.get(url)
links = set()
soup = BeautifulSoup(req.content, 'html.parser')
for atag in soup.find_all("a"):
    link = atag.get('href')
    if (link.find('https://www.baeldung.com') == -1):
        link = 'https://www.baeldung.com' + link
    links.add(link)

row = 1
for link in links:
    ws2.cell(row,column=1).value = link
    row +=1


In [40]:
url = "https://www.filmweb.pl/Powrot.Krola"
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')

filmweb = {}
filmweb['Director'] = soup.find(text="reżyseria:").next_element.find("a").text
filmweb['Year'] = soup.find("h1", class_="filmTitle").find('span').text[1:5]
boxoffice = soup.find(text="boxoffice:").next_element.text
filmweb['BoxOffice']  = re.sub(r'(top #\d+)', '', boxoffice)
filmweb['Raiting'] = soup.find("span", itemprop="ratingValue").text

row = 1
for key, value in filmweb.items():
    ws3.cell(row,column=1).value = key
    ws3.cell(row,column=2).value = value
    row += 1

In [41]:
wb.save('lab4.xlsx')